# MVD 13. cvičení
V dnešním cvičení se bude implementovat apriori a FP-Growth algoritmus pro nalezení častých vzorů.

### Úkol 1: Generování náhodných transakcí
1. Vytvořte Pandas DataFrame obsahující náhodné transakce.
    - Položky např.: ['mléko', 'chléb', 'máslo', 'vejce', 'sýr', 'pivo', 'víno', 'chipsy', 'ovoce', 'zelenina', ...]
    - Počet transakcí: Náhodně zvolte číslo mezi 50 až 100.
    - Počet položek v transakci: Každá transakce by měla obsahovat 2 až 6 náhodných položek.

2. Zobrazte prvních 5 transakcí DataFrame.

In [3]:
import pandas as pd
import numpy as np

In [35]:
items = ['mléko', 'chléb', 'máslo', 'vejce', 'sýr', 'pivo', 'víno', 'chipsy', 'ovoce', 'zelenina',]
transactions_number = np.random.randint(50, 101)  # generate random num from interval [50, 100]
items_per_transaction = np.random.random_integers(2, 6, transactions_number)  # generate `transactions_number` times random int from 2 to 6

# lambda function to randolmly choose `n` elements from list
rand_get = lambda item_list, item_num: np.random.choice(item_list, size=item_num, replace=False)
data: list[np.array,] = [rand_get(items, item_num) for item_num in items_per_transaction]

# create df
df = pd.DataFrame({'transaction': data})
df.index.name = 'id_trans'
df.head()

/var/folders/51/0prm0g2x0y13rpfcls9fpv180000gn/T/ipykernel_23434/2261488433.py:3: DeprecationWarning: This function is deprecated. Please call randint(2, 6 + 1) instead
  items_per_transaction = np.random.random_integers(2, 6, transactions_number)  # generate `transactions_number` times random int from 2 to 6


,transaction
id_trans,
0,"[zelenina, ovoce]"
1,"[sýr, pivo, vejce, víno, chipsy]"
2,"[ovoce, mléko, víno, chipsy]"
3,"[chipsy, víno, zelenina, chléb]"
4,"[chléb, víno, zelenina, mléko, chipsy, máslo]"


### Úkol 2: Implementace Apriori algoritmu
1. Napište funkci `apriori`, která:
    - Přijme DataFrame obsahující transakce.
    - Najde časté vzory (itemsety) na základě minimálního supportu (např. minsup = 0.5).
    - Vrátí seznam častých vzorů a jejich support hodnoty.

2. Otestujte funkci na vygenerovaných transakcích.

In [36]:
from itertools import combinations as itercomb

In [45]:
def apriori(df: pd.DataFrame, minsup: int = 2) -> list[str]:
    """
    Apriori algorithm.
    Find frequent itemsets (sets of items that appear together frequently in a dataset.

    Args:
        df: pandas.DataFrame with column `transaction` that contains a list of items (strings).
        minsup: - number of `support` of itemset to be considered as "frequent".

    Returns:
        all_itemsets: - list of itemsets that are considered as "frequent".
    """
    itemset_len = 1
    no_more_candidates = False
    all_itemsets = []

    candidates_sets = []
    candidates_counts = []

    while not no_more_candidates:
        for transaction in df.transaction:
            # get all item combinations from transaction with `itemset_len` length of itemset
            combinations = itercomb(transaction, itemset_len)

            # per itemset:
            for itemset in combinations:
                sorted_itemset = sorted(itemset)

                # if `sorted_itemset` already exists, increase count by 1
                if sorted_itemset in candidates_sets:
                    idx = candidates_sets.index(sorted_itemset)
                    candidates_counts[idx] += 1
                # else add it and set `count` to 1
                else:
                    candidates_sets.append(sorted_itemset)
                    candidates_counts.append(1)

        # if there is no candidates with current `itemset_len` => return current global list of itemsets (exit)
        if len(candidates_sets) == 0:
            no_more_candidates = True

        # select only itemsets with `support` value >= minimum support value
        for idx, support in enumerate(candidates_counts):
            if support >= minsup:
                all_itemsets.append((candidates_sets[idx], support))
        
        # empty all structures, increase itemset_len
        itemset_len += 1
        candidates_sets = []
        candidates_counts = []

    return all_itemsets


In [64]:
minsup = 10
result = apriori(df, minsup)

print(f"RESULTS for minsup={minsup}:\n", '-' * 20)
for itemset, support in sorted(result, key=lambda x: x[1], reverse=True):
    print(f"S={support}    {', '.join(itemset)}")

RESULTS for minsup=10:
 --------------------
S=39    víno
S=38    máslo
S=37    chipsy
S=36    zelenina
S=36    mléko
S=36    chléb
S=33    pivo
S=33    vejce
S=32    ovoce
S=30    sýr
S=20    chipsy, máslo
S=19    chipsy, víno
S=19    pivo, zelenina
S=19    mléko, vejce
S=18    víno, zelenina
S=18    chléb, zelenina
S=18    máslo, víno
S=18    mléko, máslo
S=17    vejce, víno
S=17    chipsy, ovoce
S=17    chléb, víno
S=17    chléb, mléko
S=16    chipsy, pivo
S=16    chipsy, vejce
S=16    mléko, víno
S=16    chipsy, mléko
S=16    chipsy, chléb
S=16    máslo, vejce
S=16    chléb, pivo
S=15    chipsy, sýr
S=15    chipsy, zelenina
S=15    máslo, ovoce
S=14    pivo, víno
S=14    ovoce, víno
S=14    chléb, máslo
S=14    máslo, zelenina
S=14    chléb, ovoce
S=13    ovoce, zelenina
S=13    sýr, vejce
S=13    mléko, ovoce
S=13    mléko, pivo
S=13    sýr, zelenina
S=12    pivo, vejce
S=12    máslo, sýr
S=12    mléko, sýr
S=12    ovoce, pivo
S=12    máslo, pivo
S=11    pivo, sýr
S=11    mléko, z

### Úkol 3: Implementace FP-Growth algoritmu
 1. Použijte knihovnu mlxtend k použití FP-Growth algoritmu.
 2. Převádějte DataFrame na vhodný formát pomocí TransactionEncoder.
 3. Najděte časté vzory s minimálním supportem (např. minsup = 0.5).

In [75]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth

In [76]:
def fp_growth(df: pd.DataFrame, minsup: float = 0.5):
    """
    """
    result = fpgrowth(df, min_support=minsup, use_colnames=True)
    return result


In [77]:
# change dataframe structure to fit the algorithm
te = TransactionEncoder()
dataset = df.transaction.values
te_ary = te.fit(dataset).transform(dataset)
te_df = pd.DataFrame(te_ary, columns=te.columns_)
te_df.head()

,chipsy,chléb,mléko,máslo,ovoce,pivo,sýr,vejce,víno,zelenina
0,False,False,False,False,True,False,False,False,False,True
1,True,False,False,False,False,True,True,True,True,False
2,True,False,True,False,True,False,False,False,True,False
3,True,True,False,False,False,False,False,False,True,True
4,True,True,True,True,False,False,False,False,True,True


In [84]:
result_df = fp_growth(te_df, 0.2)
result_df['support'] = result_df['support'].round(2)
result_df

,support,itemsets
0,0.44,(zelenina)
1,0.39,(ovoce)
2,0.48,(víno)
3,0.45,(chipsy)
4,0.40,(vejce)
5,0.40,(pivo)
6,0.37,(sýr)
7,0.44,(mléko)
8,0.44,(chléb)
9,0.46,(máslo)


### Úkol 4: Porovnání Apriori a FP-Growth
1. Porovnejte výsledky obou algoritmů:
    - Počet nalezených vzorů.
    - Výpočetní čas (měřte pomocí time, zkuste i zvýšit počet transakcí).